# Case study 2: How many scallop dredges do we need to sample to make sure we can detect a biologically important reduction in flatfish bycatch?


## Background
One feature that consistently emerges is that real world data are often less well-behaved than those in the studies that make it to publication. In the previous example of finfish and Nephrops separation we would seem to have a fair chance of detecting effects, if they were present.  

Onto a real world example. We will be conducting trials this year (2022) to test if Pisces can reduce numbers of flatfish - primarily Yellowtail flounder - that are caught in scallop dredges in the Northwest Atlantic. In the trial design section I illustrated some of the issues and considerations we evaluate when designing a trial. An important piece of the trial design puzzle is simulating data to ascertain how much *sampling effort* we will need to detect an effect - if it is indeed present. When there is preliminary data available, there is no excuse not to use this to make informed decisions about the trial conduct.  


````{margin}
```{attention}
I cannot stress enough the importance of either getting pilot data from some source, or simulating data using realistic published parameter estimates.  
Trials are expensive. We don't want to get to the end of a study with an equivocal result simply because we planned the 'ideal' study from an armchair, without any reality checks. 
```
````

The data set available for this trial comes from the Northeast Fisheries Observer Program (NEFOP). Fortuitously, one of their data sets is of flatfish bycatch on scallop dredges in the same closed (rather paradoxically called 'Access') area of Georges Bank that we will be working in.

````{list-table}
* - ```{figure} dupaul_team.jpg
       :height: 300px
       :align: center
     ```
  - ```{figure} scallop_dredge.jpg
       :height: 300px
       :align: center
     ```
````




## Initial exploration
The data were provided in a refreshingly nice tidy flat CSV file - checked, no data errors. The data are the weights of flatfish in each dredge, with a measure of dredge duration. We will need to read these in and convert to Catch Per Unit Effort (CPUE) to standardize the weight to a common unit. In this case, pounds (yes - pounds!) of fish per hour. Once we do this housekeeping, it's onto the important stuff.  

First things first - plot the data.  

### Are the dredges an unbiased sample device?
I mentioned above that we need to convert values to effort units - in this case, hours. This *standardizes* the counts to a common measure. However... it is wise to check that this standardization actually does what we think it's doing. Let's plot the CPUE versus effort. There should be *no relationship*. I use a log scale here to zoom up and stretch out the scales a bit. 

```{figure} yellowtaileffort1.png
:height: 300px
:align: center
```

Hang on, according to my statistics text there should be no relationship because isn't that what standardization does? 

Statistics, meet field ecology! We have higher CPUE in the shorter dredges (<30 minutes) and lower CPUE in the longer dredges (>3 hours). There are a couple of possible explanations. Short dredges might have been exploratory drops on hills or the suchlike, which might have had higher abundances of Yellowtail. With longer dredges, we might tow over less optimal habitats (shingle, maybe) which could generate a lowering of the overall average catch. Without knowing exactly what happened on the boat, these are just speculations. However, it would seem wise to restrict our exploration to a more restricted set of dredge durations. Say... 1-3 hours:  

```{figure} yellowtaileffort1.png
:height: 300px
:align: center
```

That's better. It just so happens that the decision had been made already to stick to 1 hour dredges for the trial. This would seem to be a good choice.  


### What do the catch distributions look like?
A histogram of the catches is the first point of call. As above I've converted to a log scale, simply we can't see any detail otherwise:  

```{figure} yellowtailcatch.png
:height: 300px
:align: center
```

First things first: There are a whole pile of zeroes. 922 out of 2616 data points to be exact.  
This is referred to as zero-inflation - we have more zeroes than might be expected from the theoretical statistical distributions that form the cornerstone of a lot of our analytical methods.  

````{margin}
```{note}
The more geeky types out there may be thinking to themselves "Hang on! The log of zero is undefined. How did you get it on the graph?"  
Well, my pedantic little friends, I actually plotted log10(x+0.1) for illustrative purposes. But well spotted...
```
````
Second point: Although most of the catches are around the 3 lbs/hour mark, there are a not-inconsiderable number of values up to ~200 lbs/per hour. 

Both of these in combination present analytical problems. Zeroes contain less information than a continuous scale of values, and when there are non-zero values, they are close enough to log-normally distributed.  

### Next challenge: come up with a model we can use to simulate some data
This sample distribution suggests a few alternate models. The one I chose to play with was the Tweedie distribution, which can accommodate both the zeroes and the extreme values, and appropriately scale the variance with the mean (very important!).
````{margin}
```{note}
Typically with zero-inflation we have a few models we can apply. They behave similarly so any one approach should put us in the ballpark. Really any one of these could be used:
- **Negative binomial**. Technically the negative binomial is a discrete distribution (counts), however a lot of routines to calculate a negbin model will still run. Although they'll whine and complain about it
- **Zero-inflated and hurdle models**. These are two-stage models where the zeroes are modeled as a binary process first (is there anything in the sample?), then the >0 samples modeled separately. In this data set, the non-zero values are pretty damned closed to log-normal, so we would look possibly at a hurdle log-normal model. 
   - As an aside... Zero inflated models differ from hurdle models in that the *statistical process* generating the non zero values can also conceivably generate zero values. The difference is not as discrete as for hurdles, which is why hurdle models tend to converge better than ZI (zero inflated) models
- **Tweedie models** are a relatively new addition to the statistical arsenal. They fall within the group of exponential dispersion models, and in many R implementations cover the spectrum from Poisson through Negative Binomial to inverse Gaussian
```
````
So how do we simulate a sample distribution? Let's considerable a normal or bell-shaped distribution. To generate a normal distribution I can write a simple line of code in R and tell it to output 1000 values with (in this case), a mean of 5 and standard deviation of 1:

```R
y <- rnorm(1000, 5, 1)

```

The normal distribution is described by two parameters - mean, and standard deviation. To generate a dummy data set, we need some reasonable estimates of these values.  
```{figure} normal.png
:height: 300px
:align
```
Our observed distribution is nowhere near normal, and I have already decided to use a Tweedie distribution. So what do I need to specify for that one? First, I need the Tweedie value. We can estimate this directly from the data:

```R
tweedie.profile(access$YTCPUE ~ 1, do.plot = TRUE, verbose = F)

```

```{figure} tweedie.png
:height: 300px
:align
```

The value is ~1.5, so now we can simulate a Tweedie distribution. From the data, the mean catch of Yellowtail was 3.31 lbs/hour. We will use this as our reference or control level. To generate a Tweedie distribution, we need to specify a mean value and a Tweedie parameter. WHy not a standard deviation, I hear you ask? Because the variance/standard deviation changes with the mean. The Tweedie parameter specifies the relationship of this change.  

Let's be adventurous. How many replicates do we need to sample to detect a 50% reduction in Yellowtail CPUE? We need to do a bit of coding for this step... 

```R
#Function to make our data with defined parameters
simdat <- function(n, contmean, trtmean) {
    control <- as.data.frame(rtweedie(n, xi=1.5, mu=contmean, phi=7.8))
    names(control) <- "CPUE"
    trt <- data.frame(rtweedie(n, xi=1.5, mu=trtmean, phi=7.8))
    names(trt) <- "CPUE"
    comb <- rbind(control, trt)
    Light <- c(rep("Control", n), rep("Treatment", n))
    comb2 <- data.frame(cbind(Light, comb))
    return(comb2)
}

nreps <-1000
pvalues <- numeric(nreps)

for (i in 1:nreps) {
    test <- summary(glm(CPUE ~ Light, family = tweedie(var.power = 1.5, link.power = 0),
          data=simdat(150, 3.3, 1.65)))
    pvalues[i] <- test$coefficients[2,4]
}

hist(pvalues, main="150 replicates, 50% change")
abline(v=0.05)

```
So what have I done here? I made a control distribution with a mean of 3.3, and a treatment distribution in which *Pisces* (bless its heart), generated a 50% reduction in Yellowtail catches. I specified 150 replicate dredges for ecah treament, and ran the 'experiment' 1000 times. What we want to know now, is how many times we got a 'statistically significant' result? Remember, I have already set a 'real' difference in the means. The probability of *not* finding a significant effect when there is indeed a difference is the Type II error (β). The **power** of the test is 1-β.  

By convention, most studies aim to achieve a power of 0.8. In  other words, *if* there is an effect, we should detect it 80% of the time.  
